In [25]:
import requests
import base64
import json
from secrets import *
import csv
import pandas as pd
from dotenv import load_dotenv
import os

load_dotenv()
ID = os.getenv('ID')
PSW = os.getenv('PSW')

# Step 1 - Authorization 
url = "https://accounts.spotify.com/api/token"
headers = {}
data = {}

# Encode as Base64
message = f"{ID}:{PSW}"
messageBytes = message.encode('ascii')
base64Bytes = base64.b64encode(messageBytes)
base64Message = base64Bytes.decode('ascii')


headers['Authorization'] = f"Basic {base64Message}"
data['grant_type'] = "client_credentials"

r = requests.post(url, headers=headers, data=data)

token = r.json()['access_token']


In [26]:
def query_data(params='genre:rock year:2022'):

    search_url = f"https://api.spotify.com/v1/search"

    headers = {
        "Authorization": "Bearer " + token
    }

    data = {
        'q' : params,
        'type': 'track',
        'limit': '50',
        'offset': '0',
    }

    res = requests.get(url=search_url, headers=headers, params=data)
    items = res.json()['tracks']['items']

    ids = []
    pops = []
    names = []

    for item in items :
        ids.append(item['id'])
        names.append(item['name'])
        pops.append(item['popularity'])

    return ids, pops, names

    

In [27]:
ids, pops, names = query_data(params='genre:rock year:2022')

In [28]:
df = pd.DataFrame({'id' : ids, 'popularity' : pops, 'names': names}, index=range(len(ids)))

In [29]:
df

,id,popularity,names
0,4h9wh7iOZ0GGn8QVp4RAOB,90,I Ain't Worried
1,200x8ONbPfFzByUr3fVz46,75,She Had Me At Heads Carolina
2,0HqZX76SFLDz2aW8aiqi7G,84,Bones
3,1fDFHXcykq4iw8Gg7s5hG9,80,About You
4,54ipXppHLA8U4yqpOFTUhr,82,Bones
5,6Awl7JBDQjfm7xkS3LkhDi,70,We Got History
6,58UKC45GPNTflCN6nwCUeF,71,Fancy Like
7,7ETjhPaIU4UmHOiimJBZz9,69,Your Heart Or Mine
8,72yP0DUlWPyH8P7IoxskwN,79,Hold Me Closer
9,3CIyK1V4JEJkg02E4EJnDl,78,Enemy (with JID) - from the series Arcane Leag...


In [30]:
def get_features(ids_list=ids): 
    
    headers = {
    "Authorization": "Bearer " + token
    }
    danceability_list = []
    energy_list = []
    for id_unique in ids :
        featurers_url = f"https://api.spotify.com/v1/audio-features/{id_unique}"
        data = {'id' : id_unique}
        res = requests.get(url=featurers_url, headers=headers, params=data)
        items = res.json()
        danceability_list.append(items['danceability'])
        energy_list.append(items['energy'])
    return danceability_list, energy_list

In [31]:
danceability_list, energy_list = get_features(ids_list=ids)

In [32]:
df['danceability'] = danceability_list
df['energy'] = energy_list

In [33]:
df

,id,popularity,names,danceability,energy
0,4h9wh7iOZ0GGn8QVp4RAOB,90,I Ain't Worried,0.704,0.797
1,200x8ONbPfFzByUr3fVz46,75,She Had Me At Heads Carolina,0.607,0.912
2,0HqZX76SFLDz2aW8aiqi7G,84,Bones,0.773,0.742
3,1fDFHXcykq4iw8Gg7s5hG9,80,About You,0.416,0.762
4,54ipXppHLA8U4yqpOFTUhr,82,Bones,0.772,0.750
5,6Awl7JBDQjfm7xkS3LkhDi,70,We Got History,0.628,0.833
6,58UKC45GPNTflCN6nwCUeF,71,Fancy Like,0.647,0.765
7,7ETjhPaIU4UmHOiimJBZz9,69,Your Heart Or Mine,0.639,0.713
8,72yP0DUlWPyH8P7IoxskwN,79,Hold Me Closer,0.667,0.750
9,3CIyK1V4JEJkg02E4EJnDl,78,Enemy (with JID) - from the series Arcane Leag...,0.728,0.783


In [34]:
from sklearn.linear_model import LinearRegression

from sklearn.model_selection import train_test_split

lr = LinearRegression()

X = df[['danceability','energy']]
y = df['popularity']

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7)


lr.fit(X_train,y_train)

LinearRegression()

In [35]:
lr.score(X_test,y_test)

-0.16879459259988572

In [36]:
ids_60, pops_60, names_60 = query_data(params='genre:rock year:1960')

In [37]:
danceability_list, energy_list = get_features(ids_list=ids_60)



In [38]:
df_60 = pd.DataFrame({'names': names, 'popularity' : pops_60, 'danceability' : danceability_list, 'energy' : energy_list}, index=range(len(ids)))

In [39]:
X_test = df_60[['danceability', 'energy', 'names']].set_index('names')
y_test = df_60[['popularity', 'names']].set_index('names')

In [40]:
pred = lr.predict(X_test)

In [41]:
X_test['pred'] = pred

In [42]:
X_test['pred']

names
I Ain't Worried                                                70.314053
She Had Me At Heads Carolina                                   66.753222
Bones                                                          72.443022
About You                                                      65.415457
Bones                                                          72.303605
We Got History                                                 68.339568
Fancy Like                                                     69.722439
Your Heart Or Mine                                             70.355459
Hold Me Closer                                                 70.325333
Enemy (with JID) - from the series Arcane League of Legends    70.977238
THE LONELIEST                                                  69.873077
Worth A Shot (feat. Dierks Bentley)                            67.307086
Thinking 'Bout You (feat. MacKenzie Porter)                    65.939201
Voices In My Head                            

In [43]:
lr.score(X_test, y_test)

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- pred
